In [10]:
import subprocess
import csv
import numpy as np
import matplotlib.pyplot as plt

In [134]:
mass = 2.0
velocity = 10
grade = 0.00
cadence = 80
probe_resistance = 1e-3
time_step = 1e-2
alpha = 30
rho = 1.204
CdA = 0.3


force_gravity = mass*9.81*grade
wind_force = 0.5*rho*CdA*(velocity)**2
Force = np.round(wind_force + force_gravity,8)
resistance = np.round(wind_force/velocity,8)
reactance = 1j*np.round(cadence/60*2*np.pi*mass,8)
power = np.round(Force * velocity,8)
freq = np.round(cadence/60,8)

Impedance = resistance+reactance
V_h1 = np.sqrt(2)*Force*np.sin(np.deg2rad(alpha/2))
I_h1 = np.conj(V_h1/Impedance)

left_harmonic_voltage = Force/np.sqrt(2)*(np.cos(np.radians(90))+1j*np.sin(np.radians(90)))
right_harmonic_voltage = Force/np.sqrt(2)*(np.cos(np.radians(-90+alpha))+1j*np.sin(np.radians(-90+alpha)))
net_harmonic_voltage = left_harmonic_voltage + right_harmonic_voltage
I_h1 = np.conj(net_harmonic_voltage/Impedance)
left_harmonic_power = np.real(left_harmonic_voltage*I_h1)
right_harmonic_power = np.real(right_harmonic_voltage*I_h1)


predicted_epsilon = np.real(I_h1*(left_harmonic_voltage-right_harmonic_voltage))
predicted_epsilon = Force/np.sqrt(2)*np.abs(I_h1)*np.sin(np.radians(alpha/2))*np.cos(np.radians(alpha/2 + np.degrees(np.arctan(500))-90))
predicted_epsilon -= Force/np.sqrt(2)*np.abs(I_h1)*np.sin(np.radians(alpha/2))*np.cos(np.radians(-alpha/2 + np.degrees(np.arctan(500))+90))
#predicted_epsilon = np.sqrt(2)*Force*np.cos(np.radians(alpha/2))*np.*np.sin(np.arctan(cadence/60*2*np.pi*mass))*np.abs(I_h1)
predicted_epsilon = Force**2/(np.abs(Impedance))*np.sin(np.radians(alpha))
predicted_epsilon = (Force)/(velocity*np.abs(resistance+reactance))*np.sin(np.radians(alpha))*1e2
predicted_epsilon = np.round(predicted_epsilon,4)
print(f"Predicted Error: {predicted_epsilon}%")

coeff_l = Force*np.array([0.5,1.00,0.5,0.25])
coeff_r = Force*np.array([0.5,1.00,0.5,0.25])
angle_l = np.array([0.0,90.0,180.0,-90])
angle_r = np.array([0.0,(-90.0+alpha),180.0,90])

for i in range(len(coeff_l)):
  angle_l[i] = np.radians(angle_l[i])
  angle_r[i] = np.radians(angle_r[i])


raw_spice = f'''Circuit
BinputL ing inx V={coeff_l[0]} + {coeff_l[1]}*cos({freq}*1*2*3.14159*time+{angle_l[1]})  +  {coeff_l[2]}*cos({freq}*2*2*3.14159*time+{angle_l[2]})  +  {coeff_l[3]}*cos({freq}*3*2*3.14159*time+{angle_l[3]})
BinputR inx 0  V={coeff_r[0]} + {coeff_r[1]}*cos({freq}*1*2*3.14159*time+{angle_r[1]})  +  {coeff_l[2]}*cos({freq}*2*2*3.14159*time+{angle_r[2]})  +  {coeff_r[3]}*cos({freq}*3*2*3.14159*time+{angle_r[3]})
Binputg in ing V={force_gravity*-1}
Rp p 0 {probe_resistance}
L out p {mass}
R in out {resistance}
.control
tran {time_step} 1000
run
print v(p)/{probe_resistance} v(ing) v(inx)
.endc'''
fname = f'spice/{mass}-{velocity}-{grade}-{cadence}.cir'
with open(fname, 'w') as file:
    # Write the string to the file
    file.write(raw_spice)
print("done")
# Call ngspice to run the simulation
# Call ngspice to run the simulation and capture the output
ngspice_process = subprocess.Popen(["ngspice", "-b", fname], stdout=subprocess.PIPE, text=True)
ngspice_output, _ = ngspice_process.communicate()
# Extract voltage values from ngspice output
voltage_lines =  ngspice_output.split('\n')
Time_O = []
Current_O = []
Left_Voltage_O = []
Right_Voltage_O = []
Total_Voltage_O = []
for i in voltage_lines:
  t = []
  current = []
  i = i.split("\t")
  index = i[0]
  if index.isdigit():
    Time_O.append(float(i[1]))
    Current_O.append(float(i[2]))
    Total_Voltage_O.append(float(i[3]))
    Right_Voltage_O.append(float(i[4]))
    Left_Voltage_O.append(float(i[3]) - float(i[4]))

Tsim = 50
Tstart = 900
SliceStart = int(Tstart/time_step)
SliceEnd = int((Tstart+Tsim)/time_step)

Time = Time_O[SliceStart:SliceEnd]
Current = np.array(Current_O[SliceStart:SliceEnd])
Total_Voltage = np.array(Total_Voltage_O[SliceStart:SliceEnd])
Right_Voltage = np.array(Right_Voltage_O[SliceStart:SliceEnd])
Left_Voltage = Left_Voltage_O[SliceStart:SliceEnd]
combined_array = np.column_stack((Time, Total_Voltage, Current))
np.savetxt('output.csv', combined_array, delimiter=',', header='Time,Voltage,Current', comments='')
IAV = np.round(np.sum(Current*Total_Voltage)*time_step/Tsim,3)
AAV = np.round(np.mean(Current)*np.mean(Total_Voltage),3)
right_harmonic_work = np.mean(Current*Right_Voltage)-np.mean(Current)*np.mean(Right_Voltage)
left_harmonic_work = np.mean(Current*Left_Voltage)-np.mean(Current)*np.mean(Left_Voltage)
Epsilon = np.round((right_harmonic_work-left_harmonic_work)/power*1e2,3)

print(f"Power: {AAV}")
print(f"Harmonic Power: {IAV}W")
print(f"Error: {Epsilon}%")

Predicted Error: 5.3583%
done


Note: No ".plot", ".print", or ".fourier" lines; no simulations run


Power: 180.107
Harmonic Power: 180.655W
Error: 5.308%
